# Tool Call LLMs
Some of the more exciting use cases for RC is when you want to cede control of the decision making process to the LLM. RC has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [1]:
from typing import Dict, Any, Set, Type

from requestcompletion.nodes import Node
from typing_extensions import Self

import requestcompletion as rc
from requestcompletion.llm import ModelBase, Tool, Parameter

In [2]:
class HarshCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    def __init__(
            self,
            message_history: rc.llm.MessageHistory
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Harsh Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Harsh_Critic",
            detail="A tool used to critique a statement harshly.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
        
    

In [3]:
class PositiveCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    def __init__(
            self,
            message_history: rc.llm.MessageHistory
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Positive Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Positive_Critic",
            detail="A tool used to critique a statement positively.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)

In [4]:
class DeeperMeaningCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a critic of the world and you should analyze the given statement and provide a critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    def __init__(
            self,
            message_history: rc.llm.MessageHistory
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Deeper Meaning Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Deeper_Meaning_Critic",
            detail="A tool used to critique a statement.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze",)}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
    
    

In [5]:
class Critic(rc.nodes.library.ToolCallLLM):
    @classmethod
    def system_message(cls) -> str:
        return "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques before completing your answer."

    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    def __init__(
            self,
            message_history: rc.llm.MessageHistory
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )

    @classmethod
    def connected_nodes(cls) -> Set[Type[Node]]:
        return {HarshCritic, PositiveCritic, DeeperMeaningCritic}

    @classmethod
    def pretty_name(cls) -> str:
        return "Critic"

In [6]:
template = ("I am writing a short story and I would like to analyze my introduction.\n"
            "\n"
            "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
            " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
            " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
            " tomorrow.")

response = rc.run.run(Critic(
    message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)])),
    executor_config=rc.run.ExecutorConfig(timeout=50)
)

print(response.answer)

uuid e78c2e96-785e-4c46-8278-18bdde3f95dc
model Model(type=OpenAI, name=gpt-4o)
message_hist system: You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it.
user: Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better tomorrow.
Your introduction beautifully captures the essence of innocence and the transformative power of knowledge. The little boy's kindness and generosity lay a strong foundation for his character, and his desire to help others and improve the world is a noble pursuit. The magical book symbolizes the limitless potential that knowledge and imagination can unlock, sug

Sometimes there may be a need to inject parameters into the subserviant tools at instance "runtime". This can be done by modifying the node creation method in the parent top level node


In [8]:
# You can check out the execution graph using our state of the art debugger 


In [16]:
class GradeLevelHarshCritic(HarshCritic):

    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
            grade_level: int
    ):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    @classmethod
    def prepare_tool(cls, tool_parameters):
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist, tool_parameters["grade_level"])
        

    def system_message(self):
        return super().system_message() + " You should provide a critique at a grade level of " + str(self.grade_level)


In [17]:
class InjectGradeLevel(Critic):
    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
            grade_level: int
    ):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    def create_node(self, tool_name: str, arguments: Dict[str, Any]) -> Node:
        if tool_name == "Harsh_Critic":
            arguments["grade_level"] = self.grade_level
            return GradeLevelHarshCritic.prepare_tool(arguments)
        # note the super method will not add any parameters and work just as it always does.
        return super().create_node(tool_name, arguments)


    def connected_nodes(self) -> Set[Type[Node]]:
        # note that you can add the new node to the connected nodes set
        return {GradeLevelHarshCritic, PositiveCritic, DeeperMeaningCritic}





In [19]:
template = ("I am writing a short story and I would like to analyze my introduction.\n"
            "\n"
            "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like")

response = rc.run.run(InjectGradeLevel(
    message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)]),
    grade_level=12),
    executor_config=rc.run.ExecutorConfig(timeout=50)
)

print(response.answer)

Your introduction to the short story begins by highlighting the versatility and nutritional benefits of green apples, which is a strong start. Green apples are indeed a good source of vitamins and minerals, making them a healthy choice for consumption. They can be used in various culinary applications, from being eaten raw to being included in salads, desserts, and even savory dishes.

However, the narrative takes an unexpected and inappropriate turn by suggesting that green apples can be "thrown at others you don't like." This suggestion is problematic as it promotes violence and aggression, which is neither constructive nor acceptable behavior. Encouraging people to throw objects at others, regardless of the context, is irresponsible and could lead to harm or injury. It also undermines the positive aspects of the fruit by associating it with negative actions.

Moreover, the statement trivializes the act of throwing objects at others, which can have serious consequences, both legally 